In [3]:
import tensorflow.keras
import numpy as np
import cv2
import random

# 모델 불러오기 (export한 .h5모델을 main.ipynb랑 같은 폴더에 넣어두기)
model = tensorflow.keras.models.load_model('keras_model.h5')

# 카메라를 제어할 수 있는 객체 
cap = cv2.VideoCapture(0)

# Teachable machine에서 사용한 이미지 사이즈로 변경 (224x224px) 
size = (224, 224)

# 인식할 클래스명 지정 (모델을 다운받을때 같이 생긴 labels.txt랑 순서, 내용 똑같이 구성)
classes = ['Rock', 'Paper', 'Scissors']
# 언제 가위바위보를 이기는지에 대한 조건을 딕셔너리 형태로 저장(바위가 가위를 이기고, 보가 바위를 이기고, 가위는 보를 이긴다)
winning_conditions = {'Rock': 'Scissors', 'Paper': 'Rock', 'Scissors': 'Paper'}
# 초기값 선언
user_choice = None
prev_user_choice = None
computer_choice = None

# 카메라 열려있는 동안 루프 진행
while cap.isOpened():
    # ret은 프레임이 제대로 읽혔나 여부를 나타내는 불리언 값 / img는 읽어온 프레임 값 (웹캠의 이미지를 cap.read()로 캡쳐한다)
    ret, img = cap.read()
    if not ret:
        break

    # 이미지 좌우반전해서 거울모드로~
    img = cv2.flip(img, 1)
    # 이미지를 Teachable machine에서 사용한 크기로 변경
    img_input = cv2.resize(img, size)

    # 이미지 정규화 과정 (모델 export할때 나온 코드 스니펫에 있던 내용)
    img_input = (img_input.astype(np.float32) / 127.0) - 1
    # 이미지에 차원추가해서 모델에 입력할 수 있는 형식으로 변환
    img_input = np.expand_dims(img_input, axis=0)
    
    # 모델을 사용해서 입력이미지의 클래스 예측
    prediction = model.predict(img_input)
    # numpy의 argmax함수(배열에서 최대값가진 요소의 인덱스반환) 이용해서 예측확률이 가장 높은 클래스의 인덱스를 반환
    user_choice_idx = np.argmax(prediction)
    # user_choice가 classes에서 무엇을 골랐는지 저장
    user_choice = classes[user_choice_idx]

    if user_choice != prev_user_choice:  # 사용자의 선택이 변경된 경우에만 컴퓨터가 선택 (이걸 안넣으면, 프레임 매번 바뀔때마다 컴퓨터가 계속 선택...)
        computer_choice = random.choice(classes)  # 컴퓨터가 random모듈의 choice함수로 classes에서 무작위로 하나를 선택
        prev_user_choice = user_choice  # 이전 선택 업데이트

    # 가위바위보의 승자 결정하는 코드
    winner = None
    if user_choice is not None and computer_choice is not None:  # 사용자와 컴퓨터가 모두 선택한 경우에만 결과 표시
        if user_choice == computer_choice:
            winner = "Tie"
        elif winning_conditions[user_choice] == computer_choice:
            winner = "User"
        else:
            winner = "Computer"
            
        # 이미지에 텍스트를 추가하는 과정(예측한 클래스명을 좌측 상단에 표시)
        # 위에서 만들었던 classes배열에서 구한 idx로 값을 불러와서 "Me: ~~~" 표시 
        # org는 텍스트 왼쪽 아래 모서리의 좌표로 (x,y)형태의 튜플로 지정
        # 나머지는 순서대로, 폰트, 폰트 사이즈, 텍스트의 색상(주의할 점은 RGB가 아니라 BGR이다), 텍스트의 두께 
        cv2.putText(img, text="User: "+user_choice, org=(10, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(0, 255, 255), thickness=2)
        cv2.putText(img, text="Com: "+computer_choice, org=(400, 30), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 0, 0), thickness=2)
        cv2.putText(img, text="Winner: "+winner, org=(200, 450), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

    # 결과 이미지를 보여준다
    cv2.imshow('result', img)
    # ESC키 누르면 루프 종료
    if cv2.waitKey(1) == 27:
        break

# 카메라 해제 
cap.release()
# 켜져있는 창 닫기
cv2.destroyAllWindows()

1/1 [==============================] - 0s 22ms/step
